# Information Robust Dirichlet Networks

In this notebook, we implement the specialized training loss proposed in the paper _Information Robust Dirichlet Networks for Predictive Uncertainty Estimation_ by Tsiligkaridis (2019). The method models predictive uncertainty by having a neural network output Dirichlet concentration parameters 𝛼 instead of just a pointwise softmax.

The total loss is composed of three terms:

1. Calibration term: implemented in the function  lp_fn
2. Regularization term: implemented in the function  regularization_fn
3. Adversiarial Entropy penalty: implemented in the function  dirichlet_entropy

In the paper and in this notenbook, L_p loss is not directly computed but rather an upper bound for it, denoted by F_i (for sample i)  

The regularization term penalizes high alpha values for incorrect classes.  

The final term uses the alpha values the model assigns to adversarial inputs.
The model is rewarded for outputting a Dirichlet-distribution with high entropy on these inputs.


In [ ]:
from __future__ import annotations

from typing import Optional

import torch
from torch import Tensor
from torch.special import digamma


def log_beta(a: Tensor, b: Tensor) -> Tensor:
    """Compute log(Beta(a, b))."""
    return torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a + b)


def lp_fn(alpha: Tensor, y: Tensor, p: float) -> Tensor:
    """Computes F_i (one per sample) using the expectation-based formulation.

        F_i = ( E[(1 - p_c)^p] + Σ_{j≠c} E[p_j^p] )^(1/p)

    Args:
        alpha: (B, K) Dirichlet concentration parameters (>0)
        y:     (B, K) one-hot labels
        p:     exponent scalar

    Returns:
        Sum_i F_i  (scalar Tensor)
    """
    B, K = alpha.shape  # noqa: N806

    alpha0: Tensor = alpha.sum(dim=1, keepdim=True)
    alpha_c: Tensor = (alpha * y).sum(dim=1, keepdim=True)
    alpha0_minus_c: Tensor = alpha0 - alpha_c

    # E[(1 - p_c)^p]  # noqa: ERA001
    log_e1: Tensor = log_beta(alpha0_minus_c + p, alpha_c) - log_beta(
        alpha0_minus_c,
        alpha_c,
    )
    e1: Tensor = torch.exp(log_e1)

    # E[p_j^p] for all classes
    log_ep: Tensor = log_beta(alpha + p, alpha0 - alpha) - log_beta(
        alpha,
        alpha0 - alpha,
    )
    ep: Tensor = torch.exp(log_ep)

    # zero out correct class
    ep = ep * (1 - y)

    # E1 + Σ_{j≠c} E[p_j^p]
    e_sum: Tensor = e1 + ep.sum(dim=1, keepdim=True)

    fi: Tensor = e_sum.pow(1.0 / p).squeeze(1)
    return fi.sum()


def regularization_fn(alpha: Tensor, y: Tensor) -> Tensor:
    """Implements the regularization term using trigamma-based formulation.

    Args:
        alpha: (B, K) Dirichlet parameters (>0)
        y:     (B, K) one-hot labels

    Returns:
        Scalar Tensor representing the regularization penalty.
    """
    alpha_tilde: Tensor = alpha * (1 - y) + y
    alpha_tilde_0: Tensor = alpha_tilde.sum(dim=1, keepdim=True)

    trigamma_alpha: Tensor = torch.polygamma(1, alpha_tilde)
    trigamma_alpha0: Tensor = torch.polygamma(1, alpha_tilde_0)

    diff_sq: Tensor = (alpha_tilde - 1.0).pow(2)
    mask: Tensor = 1 - y

    term: Tensor = 0.5 * diff_sq * (trigamma_alpha - trigamma_alpha0) * mask
    return term.sum(dim=1).sum()


def dirichlet_entropy(alpha: Tensor) -> Tensor:
    """Compute Dirichlet entropy for adversarial samples.

    Args:
        alpha: (B_a, K) Dirichlet parameters

    Returns:
        Scalar Tensor (sum of entropies)
    """
    K: int = alpha.shape[-1]  # noqa: N806
    alpha0: Tensor = alpha.sum(dim=-1)

    log_b: Tensor = torch.lgamma(alpha).sum(dim=-1) - torch.lgamma(alpha0)
    term1: Tensor = log_b
    term2: Tensor = (alpha0 - K) * digamma(alpha0)
    term3: Tensor = ((alpha - 1) * digamma(alpha)).sum(dim=-1)

    entropy: Tensor = term1 + term2 - term3
    return entropy.sum()


def loss_IRD(  # noqa: N802
    alpha: Tensor,
    y: Tensor,
    adversarial_alpha: Optional[Tensor] = None,  # noqa: UP045
    p: float = 2.0,
    lam: float = 1.0,
    gamma: float = 1.0,
) -> Tensor:
    """Loss from the paper.

    "Information Robust Dirichlet Networks for Predictive Uncertainty Estimation".

    Args:
        alpha: (B, K)
        y: (B, K) one-hot labels
        adversarial_alpha: (B_a, K) adversarial inputs' Dirichlet params
        p: exponent for lp-fn
        lam: weight on regularization term
        gamma: weight on entropy term (adversarial)

    Returns:
        Scalar Tensor: IRD loss
    """
    lp_term: Tensor = lp_fn(alpha, y, p)
    reg_term: Tensor = regularization_fn(alpha, y)

    if adversarial_alpha is not None:
        entropy_term: Tensor = dirichlet_entropy(adversarial_alpha)
    else:
        entropy_term = torch.tensor(0.0, device=alpha.device)

    return lp_term + lam * reg_term - gamma * entropy_term

In [34]:
# small test
alpha = torch.tensor([[2.5, 1.2, 0.8], [1.2, 4.0, 1.3]], requires_grad=True)

adversarial_alpha = torch.tensor([[5, 4, 3], [1.2, 2.0, 1.3]], requires_grad=True)

y = torch.tensor([[1, 0, 0], [0, 1, 0]], dtype=torch.float32)

loss = loss_IRD(alpha, y, adversarial_alpha)
print(loss)  # tensor of shape (B,)
loss.mean().backward()  # gradients flow properly

tensor(3.6637, grad_fn=<SubBackward0>)
